# 注意メカニズムとトランスフォーマー

リカレントネットワークの主な欠点の一つは、シーケンス内のすべての単語が結果に同じ影響を与えることです。このため、名前付きエンティティ認識や機械翻訳などのシーケンス間タスクにおいて、標準的なLSTMエンコーダーデコーダーモデルでは性能が最適化されません。実際には、入力シーケンス内の特定の単語が他の単語よりも順序出力に大きな影響を与えることがよくあります。

機械翻訳のようなシーケンス間モデルを考えてみましょう。このモデルは2つのリカレントネットワークによって実装されます。一つのネットワーク（**エンコーダー**）が入力シーケンスを隠れ状態に圧縮し、もう一つのネットワーク（**デコーダー**）がその隠れ状態を展開して翻訳結果を生成します。このアプローチの問題点は、ネットワークの最終状態が文の冒頭を記憶するのが難しくなるため、長い文に対してモデルの品質が低下することです。

**注意メカニズム**は、RNNの各出力予測に対する各入力ベクトルの文脈的影響を重み付けする手段を提供します。これを実現する方法は、入力RNNの中間状態と出力RNNの間にショートカットを作成することです。この方法では、出力記号$y_t$を生成する際に、異なる重み係数$\alpha_{t,i}$を用いてすべての入力隠れ状態$h_i$を考慮します。

![エンコーダーデコーダーモデルと加法型注意層を示す画像](../../../../../translated_images/ja/encoder-decoder-attention.7a726296894fb567.webp)
*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf)の加法型注意メカニズムを持つエンコーダーデコーダーモデル。画像は[このブログ記事](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)から引用。]*

注意行列$\{\alpha_{i,j}\}$は、出力シーケンス内の特定の単語の生成において、入力単語がどの程度影響を与えるかを表します。以下はそのような行列の例です：

![Bahdanau - arviz.orgから引用されたRNNsearch-50によるサンプルアラインメントを示す画像](../../../../../translated_images/ja/bahdanau-fig3.09ba2d37f202a6af.webp)

*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)から引用された図]*

注意メカニズムは、現在またはほぼ現在の自然言語処理の最先端技術の多くを支えています。しかし、注意を追加することでモデルのパラメータ数が大幅に増加し、RNNのスケーリング問題を引き起こしました。RNNのスケーリングにおける重要な制約は、モデルのリカレント性がバッチ処理やトレーニングの並列化を困難にすることです。RNNではシーケンスの各要素を順序通りに処理する必要があり、簡単に並列化することができません。

注意メカニズムの採用とこの制約が組み合わさり、現在の最先端技術であるトランスフォーマーモデルが誕生しました。これらはBERTからOpenGPT3まで、私たちが今日知り、使用しているモデルです。

## トランスフォーマーモデル

各予測の文脈を次の評価ステップに渡す代わりに、**トランスフォーマーモデル**は**位置エンコーディング**と注意を使用して、指定されたテキストウィンドウ内で入力の文脈を捉えます。以下の画像は、位置エンコーディングと注意が指定されたウィンドウ内で文脈を捉える方法を示しています。

![トランスフォーマーモデルで評価がどのように行われるかを示すアニメーションGIF](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

各入力位置が独立して各出力位置にマッピングされるため、トランスフォーマーはRNNよりも並列化が容易であり、より大規模で表現力豊かな言語モデルを可能にします。各注意ヘッドは、単語間の異なる関係を学習するために使用され、下流の自然言語処理タスクを改善します。

**BERT**（Bidirectional Encoder Representations from Transformers）は、非常に大規模な多層トランスフォーマーネットワークであり、*BERT-base*では12層、*BERT-large*では24層を持ちます。このモデルは、まず大規模なテキストデータ（Wikipedia + 書籍）を使用して教師なし学習（文中のマスクされた単語を予測する）で事前学習されます。事前学習中にモデルは言語理解の重要なレベルを吸収し、その後、他のデータセットで微調整することで活用できます。このプロセスは**転移学習**と呼ばれます。

![http://jalammar.github.io/illustrated-bert/から引用された画像](../../../../../translated_images/ja/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

BERT、DistilBERT、BigBird、OpenGPT3など、微調整可能なトランスフォーマーアーキテクチャには多くのバリエーションがあります。[HuggingFaceパッケージ](https://github.com/huggingface/)は、PyTorchを使用してこれらのアーキテクチャの多くをトレーニングするためのリポジトリを提供しています。

## BERTを使用したテキスト分類

事前学習済みのBERTモデルを使用して、従来のタスクであるシーケンス分類を解決する方法を見てみましょう。元のAG Newsデータセットを分類します。

まず、HuggingFaceライブラリとデータセットをロードします：


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


事前学習済みのBERTモデルを使用するため、特定のトークナイザーを使用する必要があります。まず、事前学習済みBERTモデルに関連付けられたトークナイザーを読み込みます。

HuggingFaceライブラリには事前学習済みモデルのリポジトリが含まれており、モデル名を`from_pretrained`関数の引数として指定するだけで使用できます。モデルに必要なバイナリファイルはすべて自動的にダウンロードされます。

ただし、場合によっては独自のモデルを読み込む必要があることもあります。その場合、トークナイザーのパラメータ、モデルパラメータを含む`config.json`ファイル、バイナリウェイトなど、関連するすべてのファイルを含むディレクトリを指定することができます。


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` オブジェクトには、テキストを直接エンコードするために使用できる `encode` 関数が含まれています。


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

次に、トレーニング中にデータにアクセスするために使用するイテレーターを作成しましょう。BERTは独自のエンコード関数を使用するため、以前に定義した`padify`に似たパディング関数を定義する必要があります。


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

私たちの場合、`bert-base-uncased`という事前学習済みのBERTモデルを使用します。`BertForSequenceClassfication`パッケージを使用してモデルをロードしましょう。これにより、分類のために必要なアーキテクチャがすでにモデルに含まれており、最終的な分類器も含まれています。最終的な分類器の重みが初期化されていないため、モデルには事前学習が必要であるという警告メッセージが表示されますが、それは全く問題ありません。なぜなら、まさにこれからそれを行うからです！


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

さあ、トレーニングを始めましょう！BERTはすでに事前学習されているため、初期の重みを壊さないように、学習率をかなり小さく設定して始める必要があります。

トレーニングの主要な処理は、`BertForSequenceClassification`モデルによって行われます。このモデルをトレーニングデータに適用すると、入力されたミニバッチに対して損失（loss）とネットワークの出力の両方を返します。損失はパラメータの最適化に使用され（`loss.backward()`で逆伝播を実行）、`out`は取得したラベル`labs`（`argmax`を使用して計算）と期待される`labels`を比較することでトレーニング精度を計算するために使用されます。

プロセスを管理するために、いくつかのイテレーションにわたって損失と精度を累積し、`report_freq`トレーニングサイクルごとにそれらを出力します。

このトレーニングはおそらくかなりの時間がかかるため、イテレーションの回数を制限します。


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


BERT分類を使用すると（特に反復回数を増やして十分に待つと）、かなり良い精度が得られることがわかります！これは、BERTがすでに言語の構造を非常によく理解しており、最終的な分類器を微調整するだけで済むからです。しかし、BERTは大規模なモデルであるため、トレーニング全体のプロセスには時間がかかり、かなりの計算能力（GPU、できれば複数台）が必要です！

> **Note:** この例では、最小クラスの事前学習済みBERTモデルの1つを使用しています。より大きなモデルを使用すれば、さらに良い結果が得られる可能性があります。


## モデルの性能評価

これで、テストデータセットを使ってモデルの性能を評価できます。評価ループはトレーニングループと非常に似ていますが、`model.eval()`を呼び出してモデルを評価モードに切り替えることを忘れないようにしましょう。


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## まとめ

このユニットでは、**transformers** ライブラリから事前学習済みの言語モデルを簡単に取り出し、テキスト分類タスクに適応させる方法を学びました。同様に、BERTモデルはエンティティ抽出、質問応答、その他のNLPタスクにも利用できます。

トランスフォーマーモデルは、現在のNLPの最先端を代表しており、カスタムNLPソリューションを実装する際には、まず最初に試してみるべき選択肢です。しかし、高度なニューラルモデルを構築したい場合には、このモジュールで説明したリカレントニューラルネットワークの基本的な原理を理解することが非常に重要です。



---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解釈について、当社は責任を負いません。
